In [32]:
cd("/Users/kyleconroy/desktop/ww_paper-1")

In [33]:
using Pkg
Pkg.activate("/Users/kyleconroy/desktop/ww_paper-1")

  Activating project at `~/desktop/ww_paper-1`


In [34]:
Pkg.update()
Pkg.resolve()

    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/ww_paper-1/Project.toml`
  No Changes to `~/Desktop/ww_paper-1/Manifest.toml`
  No Changes to `~/Desktop/ww_paper-1/Project.toml`
  No Changes to `~/Desktop/ww_paper-1/Manifest.toml`


In [44]:
using DrWatson
using Revise
using JLD2
using FileIO
using CSV
using DataFrames
using Turing
using DifferentialEquations
using LogExpFunctions
using Random
using ForwardDiff
using Optim
using LineSearches
using ww_paper
using Logging
using PreallocationTools

Logging.disable_logging(Logging.Warn)

LogLevel(1001)

In [45]:
## Control Parameters
n_samples = 250 #to recreate the analysis in the paper, set to 250; 10
n_chains = 4 #to recreate the anaylsis in the paper, set to 4; 1
sim = "real"
seed = 1

1

In [48]:
if sim == "real"
  dat = CSV.read(projectdir("data/Obriendata.csv"), DataFrame)
  dat = filter(:year_day => year_day -> year_day < 423, dat)
  ## Define Priors
include(projectdir("src", string("prior_constants_eirr_closed_LAdata", "_seed", seed, ".jl")))  ##Using the same constants as used in the paper

subset_dat = dat[:, [:new_time, :log_gene_copies]]
long_dat = DataFrames.stack(subset_dat, [:log_gene_copies])
long_dat = filter(:value => value -> value > 0, long_dat)
data_log_copies = long_dat[:, :value]
grid_size = 1.0
end 


1.0

In [49]:
obstimes = long_dat[:, :new_time]
obstimes = convert(Vector{Float64}, obstimes)

# pick the change times 
if maximum(obstimes) % 7 == 0
  param_change_max = maximum(obstimes) - 7
else 
  param_change_max = maximum(obstimes)
end 
param_change_times = collect(7:7.0:param_change_max)
full_time_series = collect(minimum(obstimes):grid_size:maximum(obstimes))
outs_tmp = dualcache(zeros(6,length(full_time_series)), 10)

index = zeros(length(obstimes))
for i in 1:length(index)
    time = obstimes[i]
    index[i] = indexin(round(Int64,time), full_time_series)[1]
end 

In [50]:
## Define closed form solution
include(projectdir("src/newnew_closed_soln_eirr_withincid.jl"))

## Load Model
include(projectdir("src/new_bayes_eirrc_closed.jl"))

my_model = new_bayes_eirrc_closed!(
    outs_tmp, 
    data_log_copies,
    obstimes, 
    param_change_times,
    grid_size,
    index)

DynamicPPL.Model{typeof(new_bayes_eirrc_closed!), (:outs_tmp, :data_log_copies, :obstimes, :param_change_times, :grid_size, :index), (), (), Tuple{DiffCache{Matrix{Float64}, Vector{Float64}}, Vector{Union{Missing, Float64}}, Vector{Float64}, Vector{Float64}, Float64, Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}(new_bayes_eirrc_closed!, (outs_tmp = DiffCache{Matrix{Float64}, Vector{Float64}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Any[]), data_log_copies = Union{Missing, Float64}[4.3010302, 4.7916908, 4.4199557, 4.3283796, 4.4065404, 4.5477748, 4.2013969, 4.1003704, 4.3404441, 4.6106601  …  5.4227128, 5.4233279, 5.2386732, 5.2178783, 5.4994464, 5.0543256, 5.2070956, 5.1336985, 4.825913, 5.2414842], obstimes = [1.0, 3.0, 9.0, 10.0, 15.0, 17.0, 22.0, 24.0, 29.0, 31.0  …  358.0, 360.0, 362.0, 365.0, 367.0, 369.0, 372.0, 374.0, 376.0, 37

In [51]:
function optimize_many_MAP2(model, n_reps = 100, top_n = 1, verbose = true)
    lp_res = fill(-Inf, n_reps)  # Initialize lp_res as a vector with n_reps elements, each set to -Inf
    for i in eachindex(lp_res)
        if verbose
            println(i)
        end
        Random.seed!(i)
        try
            lp_res[i] = optimize(model, MAP(), LBFGS(linesearch = LineSearches.BackTracking())).lp
        catch
        end
    end
    return lp_res
end

optimize_many_MAP2 (generic function with 4 methods)

In [52]:
Random.seed!(seed)

MAP_init = optimize_many_MAP(my_model, 10, 1, true)[1]

Random.seed!(seed)
MAP_noise = vcat(randn(length(MAP_init) - 1, n_chains), transpose(zeros(n_chains)))
MAP_noise = [MAP_noise[:,i] for i in 1:size(MAP_noise,2)]

init = repeat([MAP_init], n_chains) .+ 0.05 * MAP_noise

1
2
3
4
5
6
7
8
9
10


1-element Vector{Vector{Float64}}:
 [3.4113509613379174, -2.831777232396551, -2.2086088464379823, -0.37661802742164585, 1.2335671237861139, 0.7493576731499697, -0.39416266106959696, -0.8504178423002606, -0.16983779472455057, 0.5393016741348436  …  -0.001770910274418596, 0.031012322020963393, -0.017310649073321898, -0.05286420589243509, 1.827931971627982, 0.12165598335029328, -4.706478726466539, -2.219323480403122, 0.22374986603993396, 2.1966595579673194]

In [53]:
Random.seed!(seed)
posterior_samples = sample(my_model, NUTS(), MCMCThreads(), n_samples, n_chains, discard_initial = n_samples, init_params = init)

Chains MCMC chain (10×78×1 Array{Float64, 3}):

Iterations        = 11:1:20
Number of chains  = 1
Samples per chain = 10
Wall duration     = 10.19 seconds
Compute duration  = 10.19 seconds
parameters        = rt_params_non_centered[1], rt_params_non_centered[2], rt_params_non_centered[3], rt_params_non_centered[4], rt_params_non_centered[5], rt_params_non_centered[6], rt_params_non_centered[7], rt_params_non_centered[8], rt_params_non_centered[9], rt_params_non_centered[10], rt_params_non_centered[11], rt_params_non_centered[12], rt_params_non_centered[13], rt_params_non_centered[14], rt_params_non_centered[15], rt_params_non_centered[16], rt_params_non_centered[17], rt_params_non_centered[18], rt_params_non_centered[19], rt_params_non_centered[20], rt_params_non_centered[21], rt_params_non_centered[22], rt_params_non_centered[23], rt_params_non_centered[24], rt_params_non_centered[25], rt_params_non_centered[26], rt_params_non_centered[27], rt_params_non_centered[28], rt_params_non_ce

In [54]:
 missing_log_copies = repeat([missing], length(data_log_copies))
  
  my_model_forecast_missing = new_bayes_eirrc_closed!(
    outs_tmp, 
    missing_log_copies,
    obstimes,
    param_change_times,
    grid_size,
    index)
  
# sometimes there are NAs in the final posterior, this removes those samples
indices_to_keep = .!isnothing.(generated_quantities(my_model, posterior_samples));

posterior_samples_randn = ChainsCustomIndex(posterior_samples, indices_to_keep);

# posterior predictive output
Random.seed!(seed)
predictive_randn = predict(my_model_forecast_missing, posterior_samples_randn)

# properly scaled posterior output
Random.seed!(seed)
gq_randn = get_gq_chains(my_model, posterior_samples_randn);

CSV.write(projectdir("vignettes", string("generated_quantities", "_scenario", sim, "_seed", seed, ".csv")), DataFrame(gq_randn))
CSV.write(projectdir("vignettes", string("posterior_predictive", "_scenario", sim, "_seed", seed,  ".csv")), DataFrame(predictive_randn))

# this dataframe is saved in order to later visualize traces of the log likelihood 
posterior_df = DataFrame(posterior_samples)
CSV.write(projectdir("vignettes", string("posterior_df", "_scenario", sim, "_seed", seed, ".csv")), DataFrame(posterior_samples))

"/Users/kyleconroy/desktop/ww_paper-1/vignettes/posterior_df_scenarioreal_seed1.csv"

In [55]:
Random.seed!(seed)
prior_samples = sample(my_model, Prior(), MCMCThreads(), 400, n_chains)
    
indices_to_keep = .!isnothing.(generated_quantities(my_model, prior_samples))
prior_samples_randn = ChainsCustomIndex(prior_samples, indices_to_keep)
    
Random.seed!(seed)
prior_predictive_randn = predict(my_model_forecast_missing, prior_samples_randn)
    
Random.seed!(seed)
prior_gq_randn = get_gq_chains(my_model, prior_samples_randn)

Chains MCMC chain (400×2780×1 Array{Float64, 3}):

Iterations        = 1:1:400
Number of chains  = 1
Samples per chain = 400
parameters        = gamma, nu, eta, rho_gene, rt_init, sigma_rt, tau, lambda, df, E_init, I_init, R1_init, alpha_t_values[0], alpha_t_values[1], alpha_t_values[2], alpha_t_values[3], alpha_t_values[4], alpha_t_values[5], alpha_t_values[6], alpha_t_values[7], alpha_t_values[8], alpha_t_values[9], alpha_t_values[10], alpha_t_values[11], alpha_t_values[12], alpha_t_values[13], alpha_t_values[14], alpha_t_values[15], alpha_t_values[16], alpha_t_values[17], alpha_t_values[18], alpha_t_values[19], alpha_t_values[20], alpha_t_values[21], alpha_t_values[22], alpha_t_values[23], alpha_t_values[24], alpha_t_values[25], alpha_t_values[26], alpha_t_values[27], alpha_t_values[28], alpha_t_values[29], alpha_t_values[30], alpha_t_values[31], alpha_t_values[32], alpha_t_values[33], alpha_t_values[34], alpha_t_values[35], alpha_t_values[36], alpha_t_values[37], alpha_t_values[38]